In [1]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

In [3]:
# uninstall
!pip uninstall -y wandb

# download
!pip install transformers
!pip install simplet5

In [4]:


# import
import re
import json
import torch

import random
import pandas as pd
from tqdm import tqdm
from simplet5 import SimpleT5
from torch.utils.data import Dataset

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [5]:
# Data load function
def load_dataset():
  train_df = pd.read_csv("/content/train.dat", sep="\t",index_col=False)
  train_df.columns=["target_text","source_text"]
  train_df['source_text'] = train_df['source_text'].astype('str')
  train_df['target_text'] = train_df['target_text'].astype('str')
  train_df.reset_index(drop=True,inplace=True)

  test_df=pd.read_csv("/content/train.dat", sep="\t",index_col=False)
  test_df.columns=["target_text","source_text"]
  test_df['target_text'] = test_df['target_text'].astype('str')
  test_df['source_text'] = test_df['source_text'].astype('str')
  test_df.reset_index(drop=True,inplace=True)

  return train_df,test_df


In [6]:
train_df,test_df=load_dataset()

In [7]:
train_df.head()

,target_text,source_text
0,5,Renal abscess in children. Three cases of rena...
1,2,Hyperplastic polyps seen at sigmoidoscopy are ...
2,5,Subclavian artery to innominate vein fistula a...
3,4,Effect of local inhibition of gamma-aminobutyr...
4,1,Infection during chronic epidural catheterizat...


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14437 entries, 0 to 14436
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   target_text  14437 non-null  object
 1   source_text  14437 non-null  object
dtypes: object(2)
memory usage: 225.7+ KB


In [9]:
test_df.head()

,target_text,source_text
0,5,Renal abscess in children. Three cases of rena...
1,2,Hyperplastic polyps seen at sigmoidoscopy are ...
2,5,Subclavian artery to innominate vein fistula a...
3,4,Effect of local inhibition of gamma-aminobutyr...
4,1,Infection during chronic epidural catheterizat...


In [ ]:
from glob import glob

for trial_no in range(1):
    # create data
    train_df, test_df = load_dataset()
    # load model
    model = SimpleT5()
    model.from_pretrained(model_type="t5", model_name="t5-base")
    # train model
    model.train(train_df=train_df,
                eval_df=test_df,
                source_max_token_len=300,
                target_max_token_len=200,
                batch_size=8,
                max_epochs=2,
                outputdir = "outputs",
                use_gpu=True
               )
    # fetch the path to last model
    last_epoch_model = None
    for file in glob("./outputs/*"):
        if 'epoch-1' in file:
            last_epoch_model = file
    # load the last model
    model.load_model("t5", last_epoch_model, use_gpu=True)
    # test and save
    # for each test data perform prediction
    predictions = []
    for index, row in test_df.iterrows():
        prediction = model.predict(row['source_text'])[0]
        predictions.append(prediction)
    df = test_df.copy()
    df['predicted'] = predictions
    df['original'] = df['target_text']
    print(f1_score(df['original'], df['predicted'], average='macro'))
    df.to_csv(f"result_run_{trial_no}.csv", index=False)
    # clean the output
    #!rm -rf ./outputs

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

0.6987286014144732


INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]